<a href="https://colab.research.google.com/github/nicolascarva/Kaggle-Tanzanian-Water-Pump/blob/Full-Wrangle/Nicolas_Carvajal_DS_module_project_221.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Lambda School Data Science

*Unit 2, Sprint 2, Module 1*

---

In [67]:
import sys

# If you're on Colab:
if 'google.colab' in sys.modules:
    !pip install category_encoders==2.*
    !pip install pandas-profiling==2.*

DATA_PATH = 'https://raw.githubusercontent.com/LambdaSchool/DS-Unit-2-Kaggle-Challenge/master/data/'

In [68]:
from category_encoders import OneHotEncoder, OrdinalEncoder
from pandas_profiling import ProfileReport
from sklearn.impute import SimpleImputer
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.tree import DecisionTreeClassifier
from sklearn.tree import export_graphviz
from sklearn.feature_selection import SelectKBest
import graphviz
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
from sklearn.ensemble import RandomForestClassifier

# Decision Trees

## Kaggle

**Task 1:** [Sign up for a Kaggle account](https://www.kaggle.com/), if you don’t already have one. Go to our Kaggle InClass competition website (the URL is in Slack). Go to the Rules page. Accept the rules of the competition. Notice that the Rules page also has instructions for the Submission process. The Data page has feature definitions.

## Wrangle Data

**Task 2:** Add to the code below so that `id` is set as the index for `df`.

In [131]:
import pandas as pd

# df = pd.merge(pd.read_csv(DATA_PATH + 'waterpumps/train_features.csv'),
#               pd.read_csv(DATA_PATH + 'waterpumps/train_labels.csv'))
# YOUR CODE HERE
def wrangle(filepath1, filepath2=None):
  df_=pd.read_csv(filepath1, parse_dates= ['date_recorded'])
  
  if filepath2:
    df = pd.merge(df_,
              pd.read_csv(filepath2), on='id').set_index('id')
    
  else:
    df=df_.set_index('id')
 

 ##Change 0s in Lat and Long to NaN
  df['latitude'] = df['latitude'].mask(df['latitude'].abs() < .05, np.NaN)
  df['longitude'] = df['longitude'].mask(df['longitude'].abs() < .05, np.NaN) 
  mean_pos=df[['region_code','latitude', 'longitude']].groupby('region_code').mean()
  # long_mean_rc=df[['region_code','longitude']].groupby('region_code').mean()
  ##Replace NaNs with 
  for x in df.index.to_list():
   if df.loc[x,'latitude'] != df.loc[x,'latitude']:
     naregioncode=df.loc[x, 'region_code']
     reg_mean_lat=mean_pos.loc[naregioncode,'latitude']
     reg_mean_long=mean_pos.loc[naregioncode,'longitude']

     df.loc[x,'latitude']=reg_mean_lat
     df.loc[x,'longitude']=reg_mean_long


  # Cells dropped because of HC or only 1 value
  drop_cols_hc = [col for col in df.select_dtypes('object') 
                  if (df[col].nunique()>5000) | (df[col].nunique()==1)]

  # ['recorded_by', 'wpt_name', 'scheme_name', 'extraction_type_group','quantity_group', 'source_type', 'waterpoint_type_group']
  df=df.drop(columns=drop_cols_hc)
  print('Columns dropped because of high cardinality: ', drop_cols_hc)

  ##Drop repeated columns (trying higher cardinality first, except in columns repeated in threes (removed middle cardinality)):
  drop_cols_rep_lc = ['extraction_type','extraction_type_class','management', 'payment', 'water_quality', 'quantity',
        'source','waterpoint_type']
  drop_cols_rep_hc=['extraction_type_group', 'management_group',
       'payment_type', 'quality_group',
       'quantity_group', 'source_type', 'waterpoint_type_group']
  df=df.drop(columns=drop_cols_rep_lc)
  print('Columns dropped for being repeat: ', drop_cols_rep_lc)


  #Calculate years since construction, drop date_recorded column
  df['yrs_since_const']= (df['date_recorded'].dt.year-df['construction_year'])
  df['yrs_since_const']=df['yrs_since_const'].astype(int)
  df=df.drop(columns='date_recorded')




  
  
  return df

df=wrangle(DATA_PATH + 'waterpumps/train_features.csv',
           DATA_PATH + 'waterpumps/train_labels.csv')
X_test=wrangle(DATA_PATH + 'waterpumps/test_features.csv')
df.shape

##Calculate mean lat and long values for region code
lat_mean_rc=df[['region_code','latitude']].set_index('region_code').groupby('region_code').mean()
long_mean_rc=df[['region_code','longitude']].set_index('region_code').groupby('region_code').mean()



## for extreme cardinality, scheme name for high percentage of NaN, extraction_type_group and quantity_group, source_type for being repeated

Columns dropped because of high cardinality:  ['wpt_name', 'subvillage', 'recorded_by']
Columns dropped for being repeat:  ['extraction_type', 'extraction_type_class', 'management', 'payment', 'water_quality', 'quantity', 'source', 'waterpoint_type']
Columns dropped because of high cardinality:  ['wpt_name', 'subvillage', 'recorded_by']
Columns dropped for being repeat:  ['extraction_type', 'extraction_type_class', 'management', 'payment', 'water_quality', 'quantity', 'source', 'waterpoint_type']


In [71]:
# #Check for duplicates function - aborted

#from pandas.api.types import is_string_dtype
# from pandas.api.types import is_numeric_dtype

# for x in range(df.shape[1]):
#   col=df.iloc[:,x];
#   for y in range(x + 1, df.shape[1]):
#     otherCol = df.iloc[:, y];
#     if (is_numeric_dtype(df.iloc[:,x])) & (is_numeric_dtype(df.iloc[:,y])): 
#       dif=abs(df.iloc[:,x]-df.iloc[:,y]).mean()/abs(df.iloc[:,x].mean());
#       if dif<.1:
#         print(df.columns.values[y])
#         print(df.columns.values[x])
#       if (is_string_dtype(df.iloc[:,x])) & (is_string_dtype(df.iloc[:,y])):


In [72]:
# df[df['public_meeting'].isnull()]

,amount_tsh,date_recorded,funder,gps_height,installer,longitude,latitude,num_private,basin,region,region_code,district_code,lga,ward,population,public_meeting,scheme_management,scheme_name,permit,construction_year,extraction_type,extraction_type_group,extraction_type_class,management,management_group,payment,payment_type,water_quality,quality_group,quantity,quantity_group,source,source_type,source_class,waterpoint_type,waterpoint_type_group,status_group
id,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
8776,0.0,2013-03-06,Grumeti,1399,GRUMETI,34.698766,-2.147466,0,Lake Victoria,Mara,20,2,Serengeti,Natta,280,NaN,Other,NaN,True,2010,gravity,gravity,gravity,wug,user-group,never pay,never pay,soft,good,insufficient,insufficient,rainwater harvesting,rainwater harvesting,surface,communal standpipe,communal standpipe,functional
34169,0.0,2011-07-22,Hesawa,1162,DWE,32.920154,-1.947868,0,Lake Victoria,Mwanza,19,1,Ukerewe,Ilangala,1000,NaN,NaN,NaN,True,1999,other,other,other,vwc,user-group,never pay,never pay,milky,milky,insufficient,insufficient,spring,spring,groundwater,other,other,functional needs repair
6091,0.0,2013-02-10,Dwsp,0,DWE,33.244879,-3.491473,0,Lake Victoria,Shinyanga,17,1,Bariadi,Ikungulyabashashi,0,NaN,WUG,NaN,False,0,swn 80,swn 80,handpump,wug,user-group,unknown,unknown,unknown,unknown,unknown,unknown,shallow well,shallow well,groundwater,hand pump,hand pump,functional
24593,0.0,2013-01-22,Kkkt,1703,KKKT,35.561346,-3.806879,0,Internal,Manyara,21,3,Mbulu,Bargish,456,NaN,WUG,NaN,True,2000,nira/tanira,nira/tanira,handpump,wug,user-group,never pay,never pay,soft,good,enough,enough,shallow well,shallow well,groundwater,hand pump,hand pump,functional
35715,0.0,2013-02-27,Hesawa,1565,HESAWA,34.657034,-1.888020,0,Lake Victoria,Mara,20,2,Serengeti,Manchira,200,NaN,Other,NaN,True,1991,nira/tanira,nira/tanira,handpump,wug,user-group,other,other,soft,good,insufficient,insufficient,shallow well,shallow well,groundwater,hand pump,hand pump,non functional
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
71419,0.0,2013-01-16,Cocern,1296,TWESA,30.930385,-3.300592,0,Lake Tanganyika,Kigoma,16,1,Kibondo,Kakonko,1,NaN,NaN,Kabingo/kiobela gravity water supply,False,2013,gravity,gravity,gravity,unknown,unknown,unknown,unknown,unknown,unknown,unknown,unknown,spring,spring,groundwater,improved spring,improved spring,non functional
32815,0.0,2013-04-04,Tasaf,892,TASAF,37.744090,-4.062296,0,Pangani,Kilimanjaro,3,3,Same,Same Urban,65,NaN,NaN,NaN,NaN,0,other,other,other,unknown,unknown,unknown,unknown,unknown,unknown,unknown,unknown,unknown,other,unknown,other,other,functional
748,0.0,2013-01-22,World Vision,0,World Vision,33.244879,-3.491473,0,Lake Victoria,Shinyanga,17,1,Bariadi,Mbita,0,NaN,WUG,NaN,False,0,nira/tanira,nira/tanira,handpump,wug,user-group,unknown,unknown,soft,good,enough,enough,shallow well,shallow well,groundwater,hand pump,hand pump,functional


**Test 2**

## Split Data

**Task 3:** Create your target vector `y` and feature matrix `X`.

In [132]:
target = 'status_group'

# YOUR CODE HERE
y=df[target]
X=df.drop(columns=target)
X.shape

(59400, 28)

Train-Val Split

In [133]:
from sklearn.model_selection import train_test_split

# YOUR CODE HERE
X_train, X_val, y_train, y_val = train_test_split(X,y, test_size=.2, random_state=42)
X_train.head()

,amount_tsh,funder,gps_height,installer,longitude,latitude,num_private,basin,region,region_code,district_code,lga,ward,population,public_meeting,scheme_management,scheme_name,permit,construction_year,extraction_type_group,management_group,payment_type,quality_group,quantity_group,source_type,source_class,waterpoint_type_group,yrs_since_const
id,,,,,,,,,,,,,,,,,,,,,,,,,,,,
454,50.0,Dmdd,2092,DMDD,35.426020,-4.227446,0,Internal,Manyara,21,1,Babati,Bashinet,160,True,Water Board,NaN,True,1998,gravity,user-group,per bucket,good,insufficient,spring,groundwater,communal standpipe,15
510,0.0,Cmsr,0,Gove,35.510074,-5.724555,0,Internal,Dodoma,1,6,Bahi,Lamaiti,0,True,VWC,NaN,True,0,india mark ii,user-group,never pay,good,enough,shallow well,groundwater,hand pump,2011
14146,0.0,Kkkt,0,KKKT,32.499866,-9.081222,0,Lake Rukwa,Mbeya,12,6,Mbozi,Ndalambo,0,True,VWC,NaN,False,0,other,user-group,never pay,good,enough,shallow well,groundwater,other,2011
47410,0.0,NaN,0,NaN,34.060484,-8.830208,0,Rufiji,Mbeya,12,7,Mbarali,Chimala,0,True,VWC,NaN,True,0,gravity,user-group,monthly,good,insufficient,river/lake,surface,communal standpipe,2011
1288,300.0,Ki,1023,Ki,37.032690,-6.040787,0,Wami / Ruvu,Morogoro,5,1,Kilosa,Chakwale,120,True,VWC,NaN,True,1997,other,user-group,on failure,salty,enough,shallow well,groundwater,other,14


## Establish Baseline

**Task 5:** This is a **classification** problem, so you need to establish the baseline accuracy for your training set. Find the majority class for `y_train` and calculate the percentage of labels in `y_train` belonging to that class. Assign your answer to the variable name `baseline_acc`.

In [134]:
# YOUR CODE HERE
baseline_acc = y_train.value_counts(normalize=True).max()
print('Baseline Accuracy:', baseline_acc)

Baseline Accuracy: 0.5429713804713805


## Build Model

**Task 6:** Create a model named `model` and train it with your training data. Your model should be a pipeline with (a) transformers that you think are appropriate to this dataset and (b) a `DecisionTreeClassifier` as your predictor. **Tip:** How can you transform categorical features and missing values in order to train your model?

In [135]:
tree_model = make_pipeline(
    OrdinalEncoder(),
    SimpleImputer(strategy='median'),
    DecisionTreeClassifier(random_state=42)
)

tree_model.fit(X_train,y_train);

In [136]:
rf_model=make_pipeline(
    OrdinalEncoder(),
    SimpleImputer(strategy='median'),
    RandomForestClassifier(random_state=42)
)

rf_model.fit(X_train,y_train)

Pipeline(memory=None,
         steps=[('ordinalencoder',
                 OrdinalEncoder(cols=['funder', 'installer', 'basin', 'region',
                                      'lga', 'ward', 'public_meeting',
                                      'scheme_management', 'scheme_name',
                                      'permit', 'extraction_type_group',
                                      'management_group', 'payment_type',
                                      'quality_group', 'quantity_group',
                                      'source_type', 'source_class',
                                      'waterpoint_type_group'],
                                drop_invariant=False, handle...
                 RandomForestClassifier(bootstrap=True, ccp_alpha=0.0,
                                        class_weight=None, criterion='gini',
                                        max_depth=None, max_features='auto',
                                        max_leaf_nodes=None, max_samples=Non

## Check Metrics

**Task 7:** Calculate the training and validation accuracy of your model, and assign them to the valiables `training_acc` and `validation_acc`, respectively. Your validation accuracy should be greater than your baseline accuracy.

In [ ]:
# YOUR CODE HERE

training_acc = model.score(X_train, y_train)
validation_acc = model.score(X_val, y_val)

print('Training Accuracy:', training_acc)
print('Validation Accuracy:', validation_acc)

**Test 7**

In [ ]:
'''Task 7 Testing'''

assert isinstance(training_acc, float)
assert isinstance(validation_acc, float)
assert 0.0 <= training_acc <= 1.0
assert 0.0 <= validation_acc <= 1.0

## Create Kaggle Submission

**Task 8:** Load `'waterpumps/test.csv'` into a DataFrame named `X_test`. Generate a list of predictions, and then put them into a DataFrame `submission`. Be sure that `submission` has the same index as `X_test` and that the column name for your predictions is `'status_group'`.  

In [ ]:
# X_test = pd.read_csv(DATA_PATH + 'waterpumps/test_features.csv', index_col='id') ### This was already done
# YOUR CODE HERE

X_test=X_test
submission=pd.DataFrame(model.predict(X_test), columns=['status_group'], index=X_test.index)
submission.head(25)

**Test 8**

In [ ]:
'''Task 8 Testing'''

assert isinstance(submission, pd.DataFrame), '`submission` should be a DataFrame.'
assert len(submission) == 14358, '`submission` should have 14358 rows.'

**Task 9 (`stretch goal`):** Save `submission` as a csv file using [`.to_csv()`](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.DataFrame.to_csv.html) and submit it to our Kaggle competition.

In [ ]:
# YOUR CODE HERE


In [ ]:
from google.colab import drive
drive.mount('drive')

In [ ]:
submission.to_csv('submission.csv')
!cp "submission.csv" "drive\My Drive"

In [ ]:
submission.to_csv()

## Explain

**Task 10 (`stretch goal`):** Make a horizontal barchart of the 10 most important features for your model.

In [ ]:
import matplotlib.pyplot as plt

# YOUR CODE HERE
raise NotImplementedError()